In [ ]:
#### CODE ADOPTED FROM https://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948 ####

In [ ]:
from pandas import read_csv
from collections import Counter

import numpy as np

In [ ]:
filepath = 'dataset.csv'
column_names = ["Tweet", "Label"]

In [ ]:
# membuat list dengan isi tweet dan label dari dataset
df = read_csv(filepath, header=0, sep=',', names=column_names)
tweets = df.Tweet.to_list()
labels = df.Label.to_list()

print(df)

In [ ]:
words_ = ' '.join(tweets)
# buat list dengan isi semua kata yang ada di tweets
words = words_.split()
# hitung jumlah kata menggunakan class Counter
count_words = Counter(words)
words_count = len(count_words)
sorted_words = count_words.most_common(words_count)

In [ ]:
# tokenize words
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [ ]:
# tokenize tweets
tweets_int = []
for tweet in tweets:
    r = [vocab_to_int[w] for w in tweet.split()]
    tweets_int.append(r)
print (tweets_int[0:3])

In [ ]:
# tokenize labels
encoded_labels = [1 if label =='Setuju Vaksin' else 0 for label in labels]
encoded_labels = np.array(encoded_labels)

In [ ]:
# padding / truncating remaining data
def pad_features(tweets_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(tweets_int), seq_length), dtype = int)
    
    for i, tweet in enumerate(tweets_int):
        tweet_len = len(tweet)
        
        if tweet_len <= seq_length:
            zeroes = list(np.zeros(seq_length-tweet_len))
            new = zeroes+tweet
        elif tweet_len > seq_length:
            new = tweet[0:seq_length]
        
        features[i,:] = np.array(new)
        
    return features

features = pad_features(tweets_int, 20)
print(features[:10,:])

In [ ]:
# split dataset to train 80% validation 10% test 10%
split_frac = 0.8
train_x = features[0:int(split_frac*len(features))]
train_y = encoded_labels[0:int(split_frac*len(features))]

remaining_x = features[int(split_frac*len(features)):]
remaining_y = encoded_labels[int(split_frac*len(features)):]

valid_x = remaining_x[0:int(len(remaining_x)*0.5)]
valid_y = remaining_y[0:int(len(remaining_y)*0.5)]

test_x = remaining_x[int(len(remaining_x)*0.5):]
test_y = remaining_y[int(len(remaining_y)*0.5):]

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(valid_x), torch.from_numpy(valid_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))# dataloaders
batch_size = 50# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)